In [32]:
from iota import Iota
from pprint import pprint

In [33]:
deposit_addr = 'MPUKMWNFTYFRLJDE9ZWJY9JPKVEIIKDOWANMJIHJJWPOINFRXKVLWOUHFCMCWLO9GAAWDRWGXMTKFCIZDDQTTHNERC'
iota_node = 'https://potato.iotasalad.org:14265'

In [102]:
iota = Iota(iota_node)
iota.get_node_info()

{'latestMilestone': TransactionHash(b'VRZHTHRRIUUPCYIG9COSEQHJFQ9FXLMQUARMNYMXHHOKCBYESKWBRILQLZSOFXPRPMYWJUEQPBBQA9999'),
 'latestSolidSubtangleMilestone': TransactionHash(b'VRZHTHRRIUUPCYIG9COSEQHJFQ9FXLMQUARMNYMXHHOKCBYESKWBRILQLZSOFXPRPMYWJUEQPBBQA9999'),
 'appName': 'IRI',
 'appVersion': '1.5.3',
 'duration': 0,
 'jreAvailableProcessors': 6,
 'jreFreeMemory': 364906672,
 'jreMaxMemory': 11464605696,
 'jreTotalMemory': 3777495040,
 'jreVersion': '1.8.0_181',
 'latestMilestoneIndex': 670402,
 'latestSolidSubtangleMilestoneIndex': 670402,
 'milestoneStartIndex': 338000,
 'neighbors': 9,
 'packetsQueueSize': 0,
 'time': 1533899291707,
 'tips': 4019,
 'transactionsToRequest': 0}

In [35]:
tss = iota.find_transactions(addresses=[deposit_addr])

In [112]:
states = iota.get_latest_inclusion(tss["hashes"])

In [130]:
ths = iota.get_latest_inclusion(tss.get("hashes", []))

In [141]:
active_transactions = list(filter(lambda ts: ths["states"][ts], ths["states"]))
active_transactions

[TransactionHash(b'FWKUHIQHAGSUVNBQFMNA9ZGXLNMPDAPXLRRBKDRROHVUEAJTLPIDXWMUXIEVEFUJECAQRJZGHTFEA9999'),
 TransactionHash(b'WSDHIOKNTHJODNIF9PNHSWIYLNJCALSTD9ADHYBUFIUCHJEOBQMOTGNJGTBTBHHWSNO9XWMQVNILA9999'),
 TransactionHash(b'RBIBJBUGDGBDEBEK9KPJZRQZVTJVIFY9NBOGGXWIGYKQDUDHLMYMLDVGJVG9B9ZUSN9CRMNHHKLVZ9999')]

In [142]:
bundles = []

In [143]:
for h in active_transactions:
    b = iota.get_bundles(h)
    bundles.append(b)

In [139]:
for bundle in bundles:
    pprint(extract_json(bundle))

{'topic': 'data', 'username': 'testuser'}
{'username': 'testuser'}
{'username': 'jonas'}


In [40]:
ba = bundles[0]

In [144]:
len(bundles)

3

In [49]:
bs = ba["bundles"][0]
first_tryte_pair = bs[0].signature_message_fragment[0] + bs[0].signature_message_fragment[1]
first_tryte_pair

TryteString(b'OD')

In [145]:
bs = ba["bundles"][0]
bs

Bundle(*[{'hash_': TransactionHash(b'FWKUHIQHAGSUVNBQFMNA9ZGXLNMPDAPXLRRBKDRROHVUEAJTLPIDXWMUXIEVEFUJECAQRJZGHTFEA9999'),
         'signature_message_fragment': Fragment(b'ODGAIDGDTCFDBDPCADTCGADBEAGAHDTCGDHDIDGDTCFDGAQAEAGAHDCDDDXCRCGADBEAGASCPCHDPCGAQD99999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999

In [51]:
first_tryte_pair = bundle["bundles"][0][0].signature_message_fragment[0] + bundle["bundles"][0][0].signature_message_fragment[1]
first_tryte_pair

TryteString(b'OD')

In [92]:
from json import loads
import logging
logger = logging.getLogger()

In [99]:
def extract_json(bundle):
    extracted_json = {}
    first_transaction = bundle["bundles"][0][0]
    first_tryte_pair = first_transaction.signature_message_fragment[0] + first_transaction.signature_message_fragment[1]
    if first_tryte_pair != "OD":
        raise ValueError("No JSON found.")
    for b in bundle["bundles"]:
        for transaction in b:
            fragment = transaction.signature_message_fragment
            try:
                extracted_json.update(loads(fragment.decode()))
            except ValueError as e:
                #logger.exception(e)
                pass
    return extracted_json

In [100]:
extract_json(ba)

{'username': 'testuser', 'topic': 'data'}